In [64]:
import requests, re, pandas
from bs4 import BeautifulSoup

In [8]:
r=requests.get("http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS", 
        headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c=r.content

listings=BeautifulSoup(c, "html.parser")
# listings

all=listings.find_all('div', {'class': 'propertyRow'})

re.sub('[^0-9a-zA-Z$,]+', '', all[0].find('h4', {'class': 'propPrice'}).text)

In [78]:
l=[]
for item in all:
    d={}
    d["Locality"] = re.sub('[^0-9a-zA-Z$, ]+', '', item.find_all('span', {'class': 'propAddressCollapse'})[0].text)
    d["Address"] = re.sub('[^0-9a-zA-Z$, ]+', '', item.find_all('span', {'class': 'propAddressCollapse'})[1].text)
    d["Price"] = d[""] = re.sub('[^0-9a-zA-Z$,]+', '', item.find('h4', {'class': 'propPrice'}).text)
    try:
        d["Beds"] = re.sub('[^0-9a-zA-Z$, ]+', '', item.find('span', {'class': 'infoBed'}).find('b').text)
    except:
        d["Beds"] = None
    try:
        d["Area"] = re.sub('[^0-9a-zA-Z$, ]+', '', item.find('span', {'class': 'infoSqFt'}).find('b').text)
    except:
        d["Area"] = None
    try:
        d["Full Baths"] = re.sub('[^0-9a-zA-Z$, ]+', '', item.find('span', {'class': 'infoValueFullBath'}).find('b').text)
    except:
        d["Full Baths"] = None
    try:
        d["Half Baths"] = re.sub('[^0-9a-zA-Z$, ]+', '', item.find('span', {'class': 'infoValueHalfBath'}).find('b').text)
    except:
        d["Half Baths"] = None
        
    for column_group in item.find_all("div", {"class": "columnGroup"}):
        for feature_group, feature_name in zip(column_group.find_all("span", {"class": "featureGroup"}),
                                              column_group.find_all("span", {"class": "featureName"})):
            if "Lot Size" in feature_group.text:
                d["Lot Sixe"] = feature_name.text
    l.append(d)

In [79]:
len(l)

10

In [80]:
df=pandas.DataFrame(l)
df

,Locality,Address,Price,,Beds,Area,Full Baths,Half Baths,Lot Sixe
0,0 Gateway,"Rock Springs, WY 82901","$725,000","$725,000",None,None,None,None,NaN
1,1003 Winchester Blvd,"Rock Springs, WY 82901","$452,900","$452,900",4,None,4,None,0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901","$396,900","$396,900",5,"3,154",3,None,NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900","$389,900",4,"3,076",3,1,"Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000","$254,000",3,"1,172",3,None,"Under 1/2 Acre,"
5,1302 Veterans Drive,"Rock Springs, WY 82901","$252,900","$252,900",4,"1,932",2,None,0.27 Acres
6,1021 Cypress Cir,"Rock Springs, WY 82901","$210,000","$210,000",4,"1,676",3,None,"Under 1/2 Acre,"
7,913 Madison Dr,"Rock Springs, WY 82901","$209,000","$209,000",3,"1,344",2,None,"Under 1/2 Acre,"
8,1344 Teton Street,"Rock Springs, WY 82901","$199,900","$199,900",3,"1,920",2,None,"Under 1/2 Acre,"
9,4 Minnies Lane,"Rock Springs, WY 82901","$196,900","$196,900",3,"1,664",2,None,2.02 Acres


In [81]:
df.to_csv("Output.csv")